In [2]:
import os

path = "/kaggle/input/dig-4-bio-raman-transfer-learning-challenge"
files = os.listdir(path)
[(i, files[i]) for i in range(len(files))]

[(0, 'sample_submission.csv'),
 (1, 'timegate.csv'),
 (2, 'mettler_toledo.csv'),
 (3, 'kaiser.csv'),
 (4, 'anton_532.csv'),
 (5, 'transfer_plate.csv'),
 (6, '96_samples.csv'),
 (7, 'tornado.csv'),
 (8, 'tec5.csv'),
 (9, 'metrohm.csv'),
 (10, 'anton_785.csv')]

In [3]:
import pandas as pd

df = pd.read_csv(os.path.join(path, files[5]))
df

/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in greater
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1458: RuntimeWarning: invalid value encountered in greater
  has_large_values = (abs_vals > 1e6).any()
/usr/local/lib/python3.11/dist-packages/pandas/io/formats/format.py:1459: RuntimeWarning: invalid value encountered in less
  has_small_values = ((abs_vals < 10 ** (-self.digits)) & (abs_vals > 0)).any()
/usr/local/lib/python3.11/dist-packages/pan

,Unnamed: 0,Unnamed: 1,Unnamed: 2,Unnamed: 3,Unnamed: 4,Unnamed: 5,Unnamed: 6,Unnamed: 7,Unnamed: 8,Unnamed: 9,...,Unnamed: 2043,Unnamed: 2044,Unnamed: 2045,Unnamed: 2046,Unnamed: 2047,Unnamed: 2048,Analyte concentration,Glucose (g/L),Sodium Acetate (g/L),Magnesium Acetate (g/L)
0,sample1,[6293,7095,8325,9934,11917,14394,18925,34874,65535,...,1616,1024,1013,1067,1277,5618],sample1,4.619282,1.937172,1.052928
1,NaN,[6505,7332,8482,10175,12132,14792,19594,35813,65535,...,1655,1004,1032,1049,1271,5756],sample2,5.782718,1.175902,1.214738
2,sample2,[6478,7158,8444,9979,11932,14503,19309,35118,65535,...,1651,1024,1009,1049,1275,5685],sample3,3.953448,1.350473,2.132459
3,NaN,[6511,7308,8520,10205,12260,14777,19569,35825,65535,...,1623,1021,1008,1026,1250,5839],sample4,2.038084,0.948045,1.380240
4,sample3,[6561,7342,8562,10166,12202,14838,19593,35869,65535,...,1638,1010,1012,1047,1307,5801],sample5,4.978295,0.459765,2.539622
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
187,NaN,[6652,7453,8641,10270,12168,15014,20000,36732,65535,...,1619,1026,1025,1078,1294,5834],NaN,NaN,NaN,NaN
188,sample95,[6798,7514,8786,10431,12372,15419,20547,37854,65535,...,1587,1035,1033,1063,1271,5985],NaN,NaN,NaN,NaN
189,NaN,[6764,7534,8828,10532,12454,15504,20566,37705,65535,...,1640,1030,1006,1073,1280,5925],NaN,NaN,NaN,NaN
190,sample96,[6847,7545,8795,10452,12588,15515,20492,37710,65535,...,1667,1052,1019,1048,1266,6056],NaN,NaN,NaN,NaN


In [4]:
input_cols = df.columns[1:2049]
target_cols = df.columns[2050:]

In [5]:
targets  = df[target_cols].dropna().to_numpy()
targets[:5]

array([[4.61928175, 1.93717196, 1.0529281 ],
       [5.78271763, 1.17590196, 1.21473752],
       [3.95344763, 1.35047324, 2.13245934],
       [2.03808365, 0.94804543, 1.38023962],
       [4.97829455, 0.4597653 , 2.53962152]])

In [6]:
df = df[input_cols]
df['Unnamed: 1'] = df['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')

/tmp/ipykernel_197/3604221247.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unnamed: 1'] = df['Unnamed: 1'].str.replace('[\[\]]', '', regex=True).astype('int64')
/tmp/ipykernel_197/3604221247.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Unnamed: 2048'] = df['Unnamed: 2048'].str.replace('[\[\]]', '', regex=True).astype('int64')


In [7]:
import numpy as np

inputs = []
for i in range(0, len(df), 2):
    row1 = df.iloc[i].to_numpy()
    row2 = df.iloc[i+1].to_numpy()
    i = np.concatenate([row1, row2])
    inputs.append(i)
    
inputs = np.stack(inputs)
inputs.shape

(96, 4096)

In [8]:
from sklearn.model_selection import train_test_split


train_inputs, eval_inputs, train_targets, eval_targets = train_test_split(
        inputs,
        targets,                      
        test_size=0.2,
        random_state=1000,
)

train_inputs.shape, eval_inputs.shape, train_targets.shape, eval_targets.shape

((76, 4096), (20, 4096), (76, 3), (20, 3))

In [9]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
train_inputs = scaler.fit_transform(train_inputs)
eval_inputs = scaler.transform(eval_inputs)

In [10]:
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
from sklearn.metrics import r2_score


xgb = XGBRegressor(
    n_estimators=1000,
    max_depth=2,
    learning_rate=0.05,
    objective='reg:squarederror',
    random_state=1000,
    eval_metric='mae'
)
xgb = MultiOutputRegressor(xgb)

xgb.fit(train_inputs, train_targets, verbose=True)
preds = xgb.predict(train_inputs)
r2_score(train_targets, preds, multioutput='uniform_average')

0.9999997329524343

In [11]:
preds = xgb.predict(eval_inputs)
r2_score(eval_targets, preds, multioutput='uniform_average')

0.5801429696854837

In [12]:
test = pd.read_csv(os.path.join(path, files[6]))

row1 = test.columns[1:].to_numpy().copy()
row1[-1] = "5611"
row1 = row1.astype(np.float64)


cols = test.columns[1:]
test = test[cols]
test[" 5611]"] = test[" 5611]"].str.replace('[\[\]]', '', regex=True).astype('int64')
test = test.to_numpy()

test = np.insert(test, 0, row1, axis=0)
test = test.reshape(-1, 2, 2048).reshape(96, 2*2048)
test = test.astype(np.float32)
test.shape, test.dtype

((96, 4096), dtype('float32'))

In [13]:
test = scaler.transform(test)
preds = xgb.predict(test)

In [14]:
preds.shape

(96, 3)

In [15]:
column_names = ['Glucose', 'Sodium Acetate', 'Magnesium Sulfate']
preds_df = pd.DataFrame(preds, columns=column_names)
preds_df.insert(0, 'ID', [i+1 for i in range(len(preds_df))])
preds_df

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,3.019572,1.171645,0.757647
1,2,5.323920,1.135942,1.247337
2,3,4.121472,0.890341,0.922002
3,4,4.230670,1.093293,0.725705
4,5,7.272314,0.899426,0.886348
...,...,...,...,...
91,92,5.657174,1.855679,1.867179
92,93,3.995445,1.433699,1.123610
93,94,7.090407,1.565021,1.226262
94,95,5.584443,1.501602,1.443428


In [16]:
preds_df.to_csv("xgb_baseline_repro_1.csv", index=False)
f = pd.read_csv("/kaggle/working/xgb_baseline.csv")
f

,ID,Glucose,Sodium Acetate,Magnesium Sulfate
0,1,10.775368,1.450550,1.900435
1,2,10.223779,1.422362,2.007204
2,3,10.800874,1.420206,2.030524
3,4,10.634022,1.449118,1.870418
4,5,10.977592,1.414846,1.943152
...,...,...,...,...
91,92,4.922848,1.318014,1.197589
92,93,10.890929,1.458690,2.017366
93,94,10.721692,1.460514,1.935612
94,95,9.329219,1.461014,2.052484
